In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
# Load the dataset
df = pd.read_csv('binary_data.csv')

# Features and target
X = df[['f1', 'f2']].values
y = df['label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)


In [3]:
# Set seed for reproducibility
torch.manual_seed(0)

# Architecture: 2 input → 4 hidden → 1 output
W1 = torch.randn(2, 4, requires_grad=True)
b1 = torch.zeros(1, 4, requires_grad=True)

W2 = torch.randn(4, 1, requires_grad=True)
b2 = torch.zeros(1, 1, requires_grad=True)


In [4]:
# Binary cross entropy loss
def binary_cross_entropy(pred, target):
    epsilon = 1e-8  # avoid log(0)
    return -torch.mean(target * torch.log(pred + epsilon) + (1 - target) * torch.log(1 - pred + epsilon))

# Learning rate and epochs
lr = 0.1
epochs = 100


In [5]:
for epoch in range(1, epochs + 1):
    # Forward pass
    Z1 = X_train @ W1 + b1       # shape: (batch, 4)
    A1 = torch.relu(Z1)          # activation
    Z2 = A1 @ W2 + b2            # shape: (batch, 1)
    y_pred = torch.sigmoid(Z2)  # sigmoid output

    # Compute loss
    loss = binary_cross_entropy(y_pred, y_train)

    # Backpropagation
    loss.backward()

    # Manual update
    with torch.no_grad():
        W1 -= lr * W1.grad
        b1 -= lr * b1.grad
        W2 -= lr * W2.grad
        b2 -= lr * b2.grad

        # Zero gradients
        W1.grad.zero_()
        b1.grad.zero_()
        W2.grad.zero_()
        b2.grad.zero_()

    # Print every 10 epochs
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


Epoch 1: Loss = 0.9250
Epoch 10: Loss = 0.4917
Epoch 20: Loss = 0.3294
Epoch 30: Loss = 0.2522
Epoch 40: Loss = 0.2060
Epoch 50: Loss = 0.1755
Epoch 60: Loss = 0.1541
Epoch 70: Loss = 0.1386
Epoch 80: Loss = 0.1271
Epoch 90: Loss = 0.1182
Epoch 100: Loss = 0.1112


In [6]:
with torch.no_grad():
    # Forward on test set
    Z1_test = X_test @ W1 + b1
    A1_test = torch.relu(Z1_test)
    Z2_test = A1_test @ W2 + b2
    y_test_pred = torch.sigmoid(Z2_test)

    # Convert probabilities to labels (threshold = 0.5)
    y_test_labels = (y_test_pred >= 0.5).float()

    # Accuracy
    accuracy = (y_test_labels == y_test).float().mean() * 100
    print(f"Accuracy on test set: {accuracy:.2f}%")


Accuracy on test set: 100.00%
